In [1]:
# Created by Anastasia Filatova 
import numpy as np
import pandas as pd
import re

In [2]:
# Upload prepared data that contains all routes and all transactions
routes_df = pd.read_csv('../list_of_routes.csv', sep=',', dtype={"route_id": int, "route_type": str, 
                                                              "number": int, "additional_info": str}) 
transactions_df = pd.read_csv('../transactions_data/all_transactions.csv', sep=',', 
                              dtype={"CARD_NUM": str, "TRANSACT_DAY": int, "TRANSACT_MONTH": int, "TRANSACT_YEAR": int,
                                    "TRANSACT_TIME": str, "ID_CARRIER_SEKOP": str, "ID_ROUTE": int, "ORDER_NUM": str,
                                    "CARRIER_BOARD_NUM": int, "ID_READER": str, "TRIP_NUM": str, "CONDUCTOR_CODE": str})

In [3]:
#print("Данные обо всех коммерческих маршрутах и их id, используемых в записях транзакций")
#routes_df

In [4]:
# Оставляем только нужную для дальнешего анализа информацию в датасете
# Номер карты, id маршрута, бортовой номер машины и время транзакции (год, месяц, день, время)
#print("Данные обо всех транзакциях")
tr_df = transactions_df[["CARD_NUM", "TRANSACT_DAY", "TRANSACT_MONTH", "TRANSACT_YEAR", "TRANSACT_TIME", "ID_ROUTE", "CARRIER_BOARD_NUM"]]
#tr_df

In [5]:
# Для удобства приводим данные о времени транзакции в единый формат: hh:mm:ss
# Там, где не было информации о секундах, дописывается :00
# Туда, где информация о часах была представлена в виде h дописывается ведущий ноль
def update_time_records(times_array):
    reg1 = r"^([0-9]|0[0-9]|1[0-9]|2[0-3]):[0-5][0-9]$" #неполная запись без секунд любого вида h:mm или hh:mm
    reg2 =  r"^[0-9]:[0-5][0-9]:[0-5][0-9]$" #неполная запись часов вида 4:49:56
    for i in range(times_array.shape[0]):
        time = times_array[i]
        if re.match(reg1, time):
            time = time + ":00" 
        if re.match(reg2, time):
            time = "0" + time
        times_array[i] = time
    return times_array

In [6]:
# Достаем данные о времени транзакции из датасета и приводим в единый формат 
# с использованием вспомогательной функции update_time_records
times_array = tr_df['TRANSACT_TIME'].values
tr_df = tr_df.drop('TRANSACT_TIME', 1)
new_times_array = update_time_records(times_array)
new_time_records = pd.Series(new_times_array, index=tr_df.index)
tr_df.loc[:, 'TRANSACT_TIME'] = new_time_records
#tr_df

In [7]:
# Сортируем данные в датасете в следующем порядке: 
# год, месяц, день транзакции, id маршрута, бортовой номер машины, время транзакции
tr_df = tr_df.sort_values(by=['TRANSACT_YEAR', 'TRANSACT_MONTH', 'TRANSACT_DAY', 'ID_ROUTE', 'CARRIER_BOARD_NUM', 'TRANSACT_TIME'])
tr_df = tr_df.reset_index(drop=True)

In [8]:
# Вспомогательный блок, тут происходит подготовка данных об id маршрутов в системе СЭКОП и номеров коммерческих маршрутов
# для дальнешего анализа и установления соответствия между ними

# Создаем общий список id коммерческих маршрутов
routes_id_list = routes_df["route_id"].values.tolist()
number_of_routes = len(routes_id_list)

# Создаем общий список названий коммерческих маршрутов (с литерами, если они есть)
routes_numbers_list = []
for i in range(number_of_routes):
    liter = str(routes_df["additional_info"][i])
    if liter != "nan":
        routes_numbers_list.append(str(routes_df["number"][i]) + str(routes_df["additional_info"][i]))
    else:
        routes_numbers_list.append(str(routes_df["number"][i]))

#print("routes numbers:", routes_numbers_list)

In [9]:
# Данный блок содержит вспомогательные функции, позволяющие установить соответствие между номером маршрута 
# и его идентификатором в ситстеме СЭКОП
def get_route_id_by_num(route_num):
    for i in range(number_of_routes):
        if routes_numbers_list[i] == route_num:
            return routes_id_list[i]
    return "Error! Commercial route not found"


def get_route_num_by_id(route_id):
    for i in range(number_of_routes):
        if routes_id_list[i] == route_id:
            return routes_numbers_list[i]
    return "Error! Commercial route not found"

In [10]:
# Для удобства дальнешего анализа и визуализации в получившийся датасет добавляется информация о номере маршрута
# Для этого используются написанные вспомогательные функции
id_route_array = tr_df['ID_ROUTE'].values.tolist()
route_num = []

for i in range(len(id_route_array)):
    route_num.append(get_route_num_by_id(id_route_array[i]))
routes_numbers = pd.Series(np.array(route_num), index=tr_df.index)

In [11]:
tr_df.loc[:, 'ROUTE_NUM'] = routes_numbers
tr_df.shape[0]
# Сохраняем итог в файл
tr_df.to_csv('final_transactions_data.csv')

In [12]:
# Вспомогательная функция, вычисляющая разность между двумя переданными временными штампами в секундах 
# Допущения: timestamp2 > timestamp1, это гарантируется при подаче на вход
def get_times_delta(timestamp1, timestamp2):
    time_arr1 = timestamp1.split(':')
    time_arr2 = timestamp2.split(':')
    return int(time_arr2[2]) - int(time_arr1[2]) + 60 * (int(time_arr2[1]) - int(time_arr1[1])) + 3600 * (int(time_arr2[0]) - int(time_arr1[0]))

In [13]:
# Данный блок содержит код алгоритма, позволяющего распознать среди данных о транзакциях дубликаты
# Транзакция считается дубликатом, если она отличается от предыдущей меньше чем на 100 секунд, при этом
# все остальные параметры совпадают (дата, номер карты, номер маршрута и бортовой номер автобуса)
tr_df = tr_df.sort_values(by=['CARD_NUM', 'TRANSACT_YEAR', 'TRANSACT_MONTH', 'TRANSACT_DAY', 'TRANSACT_TIME'])
tr_df = tr_df.reset_index(drop=True)

times = tr_df['TRANSACT_TIME'].values   
board_nums = tr_df['CARRIER_BOARD_NUM'].values  
cards = tr_df['CARD_NUM'].values  
days = tr_df['TRANSACT_DAY'].values  
months = tr_df['TRANSACT_MONTH'].values  

# Составляем массив дубликатов
is_duplicate = [0]
delta = 100
delta2 = 120

for i in range(1, times.shape[0]):
    if (get_times_delta(times[i-1], times[i]) < delta2) and (get_times_delta(times[i-1], times[i]) > delta) and (cards[i] == cards[i-1]) and (board_nums[i] == board_nums[i-1]) and (days[i] == days[i-1]) and (months[i] == months[i-1]):
        print(tr_df.loc[i-1, :])
        print('-----')
        print(tr_df.loc[i, :])
        print('--------------------------------------------------------')
        is_duplicate.append(0)
    elif (get_times_delta(times[i-1], times[i]) < delta2) and (cards[i] == cards[i-1]) and (board_nums[i] == board_nums[i-1]) and (days[i] == days[i-1]) and (months[i] == months[i-1]):
        is_duplicate.append(1)
    else:
        is_duplicate.append(0)

CARD_NUM             36064299927210500
TRANSACT_DAY                        24
TRANSACT_MONTH                       9
TRANSACT_YEAR                     2018
ID_ROUTE                           979
CARRIER_BOARD_NUM                  845
TRANSACT_TIME                 22:20:51
ROUTE_NUM                         445A
Name: 88822, dtype: object
-----
CARD_NUM             36064299927210500
TRANSACT_DAY                        24
TRANSACT_MONTH                       9
TRANSACT_YEAR                     2018
ID_ROUTE                           979
CARRIER_BOARD_NUM                  845
TRANSACT_TIME                 22:22:33
ROUTE_NUM                         445A
Name: 88823, dtype: object
--------------------------------------------------------
CARD_NUM             36081385847674628
TRANSACT_DAY                        12
TRANSACT_MONTH                       9
TRANSACT_YEAR                     2018
ID_ROUTE                           631
CARRIER_BOARD_NUM                18650
TRANSACT_TIME            

In [14]:
# Добавляем поле IS_DUPLICATE в датасет. Если запись - дубликат, то значение этого поля 1, иначе 0
duplicates = pd.Series(np.array(is_duplicate), index=tr_df.index)
tr_df.loc[:, 'IS_DUPLICATE'] = duplicates
tr_df[tr_df['IS_DUPLICATE'] == 1].CARD_NUM.count() # Всего в датасете оказалось 61153 дублирующих записи

61153

In [15]:
# Сохраняем итог в файл
#tr_df.to_csv('final_transactions_with_duplicate_labels_data.csv')
tr_df[tr_df['CARD_NUM'] == '36126104505551100']

,CARD_NUM,TRANSACT_DAY,TRANSACT_MONTH,TRANSACT_YEAR,ID_ROUTE,CARRIER_BOARD_NUM,TRANSACT_TIME,ROUTE_NUM,IS_DUPLICATE
876798,36126104505551100,25,9,2018,204,433,06:01:23,88,0
876799,36126104505551100,25,9,2018,204,435,16:59:24,88,0
876800,36126104505551100,26,9,2018,200,441,06:04:52,89,0
876801,36126104505551100,27,9,2018,200,441,05:59:33,89,0
876802,36126104505551100,27,9,2018,200,438,16:59:39,89,0
876803,36126104505551100,27,9,2018,200,438,17:01:25,89,0
876804,36126104505551100,1,10,2018,200,441,05:59:18,89,0
876805,36126104505551100,1,10,2018,200,203,17:53:43,89,0
876806,36126104505551100,3,10,2018,204,2,06:01:12,88,0
876807,36126104505551100,4,10,2018,200,430,17:10:55,89,0


In [3]:
df_routes_by_stops = pd.read_csv('RBS.csv', ';')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xca in position 0: invalid continuation byte